In [4]:
# the below code is from DAE course notebook, so it needs to be altered to avoid plagiarism

import numpy as np  # import auxiliary library, typical idiom
import pandas as pd  # import the Pandas library, typical idiom

# next command ensures that plots appear inside the notebook
# %matplotlib inline
# import matplotlib as mpl
# import matplotlib.pyplot as plt
# import seaborn as sns  # also improves the look of plots
# sns.set()
# plt.rcParams['figure.figsize'] = 10, 5  # default hor./vert. size of plots, in inches
# plt.rcParams['lines.markeredgewidth'] = 1  # to fix issue with seaborn box plots; needed after import seaborn

# # reveal a hint only while holding the mouse down
# from IPython.display import HTML
# HTML("<style>.h,.c{display:none}.t{color:#296eaa}.t:active+.h{display:block;}</style>")

In [29]:
from datetime import datetime
from numba import jit

In [52]:
# the below code is from DAE course notebook, so it needs to be altered to avoid plagiarism

from sklearn.linear_model import LinearRegression  # for linear regression
from sklearn.cluster import KMeans  # for clustering
from sklearn.tree import DecisionTreeClassifier  # for decision tree mining
from sklearn.metrics import mean_absolute_error, confusion_matrix
from sklearn.model_selection import train_test_split
#from treeviz import tree_print  # to print decision tree

In [2]:
file_export = 'export2012.csv'

In [5]:
data = pd.read_csv(file_export)

In [6]:
data.head(10)

,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource
0,173688,A_SUBMITTED,2011/10/01 00:38:44.546,2011/10/01 00:38:44.546,20000,2011/10/01 00:38:44.546,112
1,173688,A_PARTLYSUBMITTED,2011/10/01 00:38:44.880,2011/10/01 00:38:44.880,20000,2011/10/01 00:38:44.546,112
2,173688,A_PREACCEPTED,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112
3,173688,A_PREACCEPTED,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112
4,173688,W_Completeren aanvraag,2011/10/01 11:36:46.437,2011/10/01 11:45:13.917,20000,2011/10/01 00:38:44.546,112
5,173688,A_ACCEPTED,2011/10/01 11:42:43.308,2011/10/01 11:42:43.308,20000,2011/10/01 00:38:44.546,10862
6,173688,O_SELECTED,2011/10/01 11:45:09.243,2011/10/01 11:45:09.243,20000,2011/10/01 00:38:44.546,10862
7,173688,A_FINALIZED,2011/10/01 11:45:09.243,2011/10/01 11:45:09.243,20000,2011/10/01 00:38:44.546,10862
8,173688,O_CREATED,2011/10/01 11:45:11.197,2011/10/01 11:45:11.197,20000,2011/10/01 00:38:44.546,10862
9,173688,O_SENT,2011/10/01 11:45:11.380,2011/10/01 11:45:11.380,20000,2011/10/01 00:38:44.546,10862


In [87]:
df.to_csv('data.csv') # allows to export the dataframe as a csv file

In [7]:
data.describe()

,case,AMOUNT_REQ,org:resource
count,190827.000000,190827.000000,190827.000000
mean,193851.453673,15438.929088,8271.081299
std,11688.517781,12315.133412,4700.713383
min,173688.000000,0.000000,112.000000
25%,183292.000000,6000.000000,10138.000000
50%,194242.000000,10500.000000,10913.000000
75%,203850.000000,20000.000000,11121.000000
max,214376.000000,99999.000000,11339.000000


In [8]:
data_sorted = data.sort_values(by=['case', 'event','completeTime'])

In [9]:
data_sorted.head(10)

,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource
5,173688,A_ACCEPTED,2011/10/01 11:42:43.308,2011/10/01 11:42:43.308,20000,2011/10/01 00:38:44.546,10862
20,173688,A_ACTIVATED,2011/10/13 10:37:29.226,2011/10/13 10:37:29.226,20000,2011/10/01 00:38:44.546,10629
18,173688,A_APPROVED,2011/10/13 10:37:29.226,2011/10/13 10:37:29.226,20000,2011/10/01 00:38:44.546,10629
7,173688,A_FINALIZED,2011/10/01 11:45:09.243,2011/10/01 11:45:09.243,20000,2011/10/01 00:38:44.546,10862
1,173688,A_PARTLYSUBMITTED,2011/10/01 00:38:44.880,2011/10/01 00:38:44.880,20000,2011/10/01 00:38:44.546,112
2,173688,A_PREACCEPTED,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112
3,173688,A_PREACCEPTED,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112
17,173688,A_REGISTERED,2011/10/13 10:37:29.226,2011/10/13 10:37:29.226,20000,2011/10/01 00:38:44.546,10629
0,173688,A_SUBMITTED,2011/10/01 00:38:44.546,2011/10/01 00:38:44.546,20000,2011/10/01 00:38:44.546,112
19,173688,O_ACCEPTED,2011/10/13 10:37:29.226,2011/10/13 10:37:29.226,20000,2011/10/01 00:38:44.546,10629


In [10]:
df = data.copy()

# convert to time format the string type time. This will allow to calculate the time duration of the task.
df["startTime"] = pd.to_datetime(df["startTime"])
df["completeTime"] = pd.to_datetime(df["completeTime"])
df["duration"] = df["completeTime"] - df["startTime"]
df

,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource,duration
0,173688,A_SUBMITTED,2011-10-01 00:38:44.546,2011-10-01 00:38:44.546,20000,2011/10/01 00:38:44.546,112,0 days 00:00:00
1,173688,A_PARTLYSUBMITTED,2011-10-01 00:38:44.880,2011-10-01 00:38:44.880,20000,2011/10/01 00:38:44.546,112,0 days 00:00:00
2,173688,A_PREACCEPTED,2011-10-01 00:39:37.906,2011-10-01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,0 days 00:00:00
3,173688,A_PREACCEPTED,2011-10-01 00:39:37.906,2011-10-01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,0 days 00:00:00
4,173688,W_Completeren aanvraag,2011-10-01 11:36:46.437,2011-10-01 11:45:13.917,20000,2011/10/01 00:38:44.546,112,0 days 00:08:27.480000
...,...,...,...,...,...,...,...,...
190822,214376,A_SUBMITTED,2012-02-29 23:51:16.799,2012-02-29 23:51:16.799,15000,2012/02/29 23:51:16.799,112,0 days 00:00:00
190823,214376,A_PARTLYSUBMITTED,2012-02-29 23:51:17.423,2012-02-29 23:51:17.423,15000,2012/02/29 23:51:16.799,112,0 days 00:00:00
190824,214376,A_PARTLYSUBMITTED,2012-02-29 23:51:17.423,2012-02-29 23:51:17.423,15000,2012/02/29 23:51:16.799,112,0 days 00:00:00
190825,214376,W_Afhandelen leads,2012-03-01 09:26:46.736,2012-03-01 09:27:41.325,15000,2012/02/29 23:51:16.799,11169,0 days 00:00:54.589000


In [85]:
#datetime.strptime(data["completeTime"], '%Y-%m-%d')
#print(datetime)

In [11]:
data['startTime'] == data["completeTime"]

0          True
1          True
2          True
3          True
4         False
          ...  
190822     True
190823     True
190824     True
190825    False
190826     True
Length: 190827, dtype: bool

The above shows that the startTime and completeTime are not always the same.

In [12]:
data_sorted2 = data.sort_values(by=['AMOUNT_REQ'])

In [13]:
data_sorted2

,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource
91296,193378,W_Afhandelen leads,2011/12/14 09:19:24.585,2011/12/14 09:28:18.936,0,2011/12/13 21:29:24.105,11189
91300,193378,A_CANCELLED,2011/12/14 09:43:25.393,2011/12/14 09:43:25.393,0,2011/12/13 21:29:24.105,10779
91299,193378,W_Afhandelen leads,2011/12/14 09:43:25.385,2011/12/14 09:43:25.385,0,2011/12/13 21:29:24.105,11189
91298,193378,W_Afhandelen leads,2011/12/14 09:32:45.306,2011/12/14 09:35:13.883,0,2011/12/13 21:29:24.105,11189
91293,193378,A_SUBMITTED,2011/12/13 21:29:24.106,2011/12/13 21:29:24.106,0,2011/12/13 21:29:24.105,112
...,...,...,...,...,...,...,...
179483,211643,A_SUBMITTED,2012/02/22 08:07:14.767,2012/02/22 08:07:14.767,99999,2012/02/22 08:07:14.767,112
49046,183576,A_SUBMITTED,2011/11/09 19:47:51.883,2011/11/09 19:47:51.883,99999,2011/11/09 19:47:51.882,112
49048,183576,A_DECLINED,2011/11/09 19:47:54.236,2011/11/09 19:47:54.236,99999,2011/11/09 19:47:51.882,112
179486,211643,A_PREACCEPTED,2012/02/22 08:07:58.846,2012/02/22 08:07:58.846,99999,2012/02/22 08:07:14.767,112


In [90]:
# data["next_task"] = calculator_nb(data)

data_iterrated = data.copy()
# for i in data_iterrated.iterrows():
#     if (i[0]+1 >= len(data)):
#         break
#     data["next_task"] = data.at[i[0]+1, 'case'] if (data.at[i[0] + 1, "case"] == i[1]["case"]) else 'null'

@jit(nopython=True, parallel=True)
def calculator_nb(case):
    res = np.empty(len(case))
    idx = 0
    for _ in case:
        # print(idx)
        if (idx+1 >= len(case)):
            break
        if (case[idx + 1] != case[idx]):
            res[idx] = case[idx + 1]
        else:
            res[idx] = 0

        idx+=1
    return res


# print(data_iterrated['case'].array)
data_iterrated["next_task"] = calculator_nb(data_iterrated['case'].values)

C:\Python310\lib\site-packages\numba\core\typed_passes.py:329: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.readthedocs.io/en/stable/user/parallel.html#diagnostics for help.

File "..\..\..\AppData\Local\Temp\ipykernel_8100\2411862996.py", line 9:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaPerformanceWarning(msg,


In [91]:
data_iterrated.head(40)

,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource,next_task
0,173688,A_SUBMITTED,2011/10/01 00:38:44.546,2011/10/01 00:38:44.546,20000,2011/10/01 00:38:44.546,112,0.0
1,173688,A_PARTLYSUBMITTED,2011/10/01 00:38:44.880,2011/10/01 00:38:44.880,20000,2011/10/01 00:38:44.546,112,0.0
2,173688,A_PREACCEPTED,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,0.0
3,173688,A_PREACCEPTED,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,0.0
4,173688,W_Completeren aanvraag,2011/10/01 11:36:46.437,2011/10/01 11:45:13.917,20000,2011/10/01 00:38:44.546,112,0.0
5,173688,A_ACCEPTED,2011/10/01 11:42:43.308,2011/10/01 11:42:43.308,20000,2011/10/01 00:38:44.546,10862,0.0
6,173688,O_SELECTED,2011/10/01 11:45:09.243,2011/10/01 11:45:09.243,20000,2011/10/01 00:38:44.546,10862,0.0
7,173688,A_FINALIZED,2011/10/01 11:45:09.243,2011/10/01 11:45:09.243,20000,2011/10/01 00:38:44.546,10862,0.0
8,173688,O_CREATED,2011/10/01 11:45:11.197,2011/10/01 11:45:11.197,20000,2011/10/01 00:38:44.546,10862,0.0
9,173688,O_SENT,2011/10/01 11:45:11.380,2011/10/01 11:45:11.380,20000,2011/10/01 00:38:44.546,10862,0.0


In [58]:
data_iterrated.head()

,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource,next_task
0,173688,A_SUBMITTED,2011/10/01 00:38:44.546,2011/10/01 00:38:44.546,20000,2011/10/01 00:38:44.546,112,1
1,173688,A_PARTLYSUBMITTED,2011/10/01 00:38:44.880,2011/10/01 00:38:44.880,20000,2011/10/01 00:38:44.546,112,1
2,173688,A_PREACCEPTED,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,1
3,173688,A_PREACCEPTED,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,1
4,173688,W_Completeren aanvraag,2011/10/01 11:36:46.437,2011/10/01 11:45:13.917,20000,2011/10/01 00:38:44.546,112,1
